In [2]:
import requests
import pandas as pd
import re

response = requests.get('https://www.gutenberg.org/cache/epub/100/pg100.txt')

text = response.text

para = list()

for i in text.split("THE SONNETS")[2].split("\r\n"):
  para.append(i)

shakes = pd.DataFrame(para)

shakes.head(100)

shakes = shakes.rename(columns={0:"words"})

shakes['words'] = shakes.words.apply(lambda x: x.strip())

shakes = shakes[shakes['words'] != ""]

shakes['words'] = shakes.words.apply(lambda x: re.sub("[0-9]\r\n","", x))
shakes['words'] = shakes.words.apply(lambda x: re.sub("[0-9]","", x))
shakes['words'] = shakes.words.apply(lambda x: re.sub("[,-\[\]'$({})]","",x))
shakes['words'] = shakes.words.apply(lambda x: x.replace("\r", ""))
shakes['words'] = shakes.words.apply(lambda x: x.replace("\n", ""))

lines = [line for line in list(shakes.words) if line != ""]


print(shakes.head(10))

                                                words
2                                                    
4            rom fairest creatures we desire increase
5           hat thereby beauty’s rose might never die
6              ut as the riper should by time decease
7                is tender heir might bear his memory
8         ut thou contracted to thine own bright eyes
9   eed’st thy light’s flame with selfsubstantial ...
10                aking a famine where abundance lies
11         hyself thy foe to thy sweet self too cruel
12        hou that art now the world’s fresh ornament


In [3]:
import tqdm
import spacy
nlp = spacy.load("en_core_web_sm")
# nlp.max_length = 9999999

def text_pipeline_spacy(lines):
    p_tokens = []
    
    #beware!!!! this ate my laptop resources... rerun this at risk!!!!! (I personally wouldn't risk it!!!) ~Team18
    doc = nlp(lines)
    for t in doc:
        if not t.is_stop and not t.is_punct and not t.is_space:
            p_tokens.append(t.lemma_.lower())
    return p_tokens

lines = [text_pipeline_spacy(line) for line in tqdm.tqdm(lines)]

len(lines)

100%|█████████████████████████████████████████████████████████████████████████| 124309/124309 [10:23<00:00, 199.33it/s]


124309

In [74]:
# # !conda install gensim
# import gensim
# from gensim.utils import  simple_preprocess

# ngram2 = gensim.models.Phrases(lines, min_count=1, threshold=1)
# ngram3 = gensim.models.Phrases(ngram2[lines], min_count=2, threshold=1)
# ngram4 = gensim.models.Phrases(ngram3[ngram2[lines]], min_count=1, threshold=1)
# ngram5 = gensim.models.Phrases(ngram4[ngram3[ngram2[lines]]], min_count=2, threshold=1)
# ngram2_phraser = gensim.models.phrases.Phraser(ngram2)
# ngram3_phraser = gensim.models.phrases.Phraser(ngram3)
# ngram4_phraser = gensim.models.phrases.Phraser(ngram4)
# ngram5_phraser = gensim.models.phrases.Phraser(ngram5)


In [75]:
# def bigramsfn(lines):
#     lines = [ngram2_phraser[line] for line in lines]
#     return lines

# ngrams2_words = bigramsfn(lines)

In [76]:
# def trigramsfn(lines):
#     lines = [ngram3_phraser[ngram2_phraser[line]] for line in lines]
#     return lines

# ngrams3_words = trigramsfn(bigrams)

In [77]:
from nltk import ngrams

stopwords = ["", "™"]

words = [word for line in lines for word in line if word not in stopwords]

for i in range(2,6):
    ngram_counts = Counter(ngrams(words, i))
    print("at ngrams =",i)
    print(ngram_counts.most_common(10))
    print("----------")

at ngrams = 2
[(('thou', 'art'), 432), (('y', 'lord'), 344), (('thou', 'hast'), 276), (('o', 'th'), 257), (('thou', 'shalt'), 182), (('good', 'lord'), 182), (('f', 'thou'), 180), (('art', 'thou'), 175), (('hou', 'art'), 161), (('ir', 'ohn'), 156)]
----------
at ngrams = 3
[(('y', 'good', 'lord'), 38), (('hy', 'dost', 'thou'), 34), (('hat', 'art', 'thou'), 30), (('ve', 'thy', 'hand'), 27), (('hat', 'wouldst', 'thou'), 25), (('o', 'good', 'lord'), 24), (('hat', 'dost', 'thou'), 24), (('ir', 'ohn', 'alstaff'), 24), (('thee', 'thou', 'art'), 23), (('nd', 'thou', 'shalt'), 22)]
----------
at ngrams = 4
[(('roject', 'utenberg', 'electronic', 'work'), 18), (('roject', 'utenberg', 'iterary', 'rchive'), 13), (('utenberg', 'iterary', 'rchive', 'oundation'), 13), (('elmont', 'room', 'ortia', 'house'), 12), (('ossillon', 'room', 'ountess', 'palace'), 10), (('cene', 'lexandria', 'oom', 'alace'), 8), (('lexandria', 'oom', 'alace', 'cene'), 8), (('lexandria', 'oom', 'alace', 'nter'), 8), (('xeunt', '